In [1]:
import math
import os
import requests
import numpy as np
import torch

import matplotlib.pyplot as plt
from IPython.display import Audio, display

import torchaudio
torchaudio.set_audio_backend("soundfile")

In [2]:
PWD_PATH = 'D:\\ML\\Speech recognition\\NLP_diploma\\ASR proto\\ASR proto\\notebooks'
print(PWD_PATH)

D:\ML\Speech recognition\NLP_diploma\ASR proto\ASR proto\notebooks


In [3]:
_SAMPLE_DIR = "_sample_dir"

SAMPLE_WAV_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/steam-train-whistle-daniel_simon.wav"
SAMPLE_WAV_PATH = os.path.join(_SAMPLE_DIR, "steam.wav")

SAMPLE_RIR_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/distant-16k/room-response/rm1/impulse/Lab41-SRI-VOiCES-rm1-impulse-mc01-stu-clo.wav"
SAMPLE_RIR_PATH = os.path.join(_SAMPLE_DIR, "rir.wav")

SAMPLE_WAV_SPEECH_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/source-16k/train/sp0307/Lab41-SRI-VOiCES-src-sp0307-ch127535-sg0042.wav"
SAMPLE_WAV_SPEECH_PATH = os.path.join(_SAMPLE_DIR, "speech.wav")

SAMPLE_NOISE_URL = "https://pytorch-tutorial-assets.s3.amazonaws.com/VOiCES_devkit/distant-16k/distractors/rm1/babb/Lab41-SRI-VOiCES-rm1-babb-mc01-stu-clo.wav"
SAMPLE_NOISE_PATH = os.path.join(_SAMPLE_DIR, "bg.wav")

os.makedirs(_SAMPLE_DIR, exist_ok=True)

def _fetch_data():
    uri = [
        (SAMPLE_WAV_URL, SAMPLE_WAV_PATH),
        (SAMPLE_RIR_URL, SAMPLE_RIR_PATH),
        (SAMPLE_WAV_SPEECH_URL, SAMPLE_WAV_SPEECH_PATH),
        (SAMPLE_NOISE_URL, SAMPLE_NOISE_PATH),
    ]
    for url, path in uri:
        with open(path, 'wb') as file_:
            file_.write(requests.get(url).content)

_fetch_data()

In [4]:
import soundfile as sf



# Extract audio data and sampling rate from file 
data, fs = sf.read(SAMPLE_NOISE_PATH) 
print(f"{data.shape=}")
print(f"{fs=}")

if len(data.shape) == 1:
    display(Audio(data, rate=fs))
else:
    data = data.T
    display(Audio((data[0], data[1]), rate=fs))

data.shape=(80000,)
fs=16000


In [6]:
def _get_sample(path, resample=None):
    """
    effects = [
        ["remix", "1"]
    ]
    if resample:
        effects.extend([
          ["lowpass", f"{resample // 2}"],
          ["rate", f'{resample}'],
        ])
    """
    data, fs = sf.read(SAMPLE_NOISE_PATH) 
    return data, fs


def get_noise_sample(*, resample=None):
    return _get_sample(SAMPLE_NOISE_PATH, resample=resample)
    
def get_speech_sample(*, resample=None):
    return _get_sample(SAMPLE_WAV_SPEECH_PATH, resample=resample)


def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
    waveform = waveform.numpy() if not type(waveform) is np.ndarray else waveform

    num_channels = 1
    num_frames = waveform.shape[0]
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].specgram(waveform[c], Fs=sample_rate)
        if num_channels > 1:
            axes[c].set_ylabel(f'Channel {c+1}')
        if xlim:
            axes[c].set_xlim(xlim)
    figure.suptitle(title)
    plt.show(block=False)

    
def play_audio(waveform, sample_rate):
    waveform = waveform.numpy() if not type(waveform) is np.ndarray else waveform

    num_channels = 1
    num_frames = waveform.shape[0]
    if num_channels == 1:
        display(Audio(waveform, rate=sample_rate))
    elif num_channels == 2:
        display(Audio((waveform[0], waveform[1]), rate=sample_rate))
    else:
        raise ValueError("Waveform with more than 2 channels are not supported.")
        

def get_rir_sample(*, resample=None, processed=False):
    rir_raw, sample_rate = _get_sample(SAMPLE_RIR_PATH, resample=resample)
    if not processed:
        return rir_raw, sample_rate
    rir = rir_raw[:, int(sample_rate*1.01):int(sample_rate*1.3)]
    rir = rir / torch.norm(rir, p=2)
    rir = torch.flip(rir, [1])
    return rir, sample_rate

In [7]:
sample_rate = 8000
speech, _ = get_speech_sample(resample=sample_rate)
noise, _ = get_noise_sample(resample=sample_rate)
#noise = noise[:, :speech.shape[1]]

#plot_waveform(noise, sample_rate, title="Background noise")
#plot_specgram(noise, sample_rate, title="Background noise")
play_audio(speech, sample_rate)
play_audio(noise, sample_rate)

speech_power = np.linalg.norm(speech)
noise_power = np.linalg.norm(noise)

for snr_db in [20, 10, 3]:
    snr = math.exp(snr_db / 10)
    scale = snr * noise_power / speech_power
    noisy_speech = (scale * speech + noise) / 2

    #plot_waveform(noisy_speech, sample_rate, title=f"SNR: {snr_db} [dB]")
    #plot_specgram(noisy_speech, sample_rate, title=f"SNR: {snr_db} [dB]")
    play_audio(noisy_speech, sample_rate)


In [36]:
a = torch.arange(0, 10) / 10

In [52]:
np.linalg.norm([10, 10 ,9])

16.76305461424021